<a href="https://colab.research.google.com/github/Russo-Federico/DeepLearningFundamentals/blob/main/IBM-PyTorch-DL/0-PyTorchBasicsForDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Neural Network

In [25]:
import numpy as np
import torch
import torch.nn as nn
from torch import sigmoid

Create a NN class

In [6]:
class NN(nn.Module):
  # D_in - input size of the network
  # H - number of neurons
  # D_out - output size of the netowrk
  def __init__(self, D_in, H, D_out):
    super(NN, self).__init__()
    self.linear1 = nn.Linear(D_in, H) # input layer
    self.linear2 = nn.Linear(H, D_out) # output layer

  def forward(self, x):
    x = sigmoid(self.linear1(x))
    x = sigmoid(self.linear2(x))
    return x

In [7]:
x = torch.tensor([1.0])

model = NN(1,2,1)
yhat = model(x)
print("yhat:", yhat)

yhat: tensor([0.6091], grad_fn=<SigmoidBackward0>)


In [8]:
model.state_dict()

OrderedDict([('linear1.weight',
              tensor([[-0.5834],
                      [-0.4041]])),
             ('linear1.bias', tensor([-0.5011,  0.8688])),
             ('linear2.weight', tensor([[0.5401, 0.2700]])),
             ('linear2.bias', tensor([0.1411]))])

Build a network using the Sequential module

In [9]:
seq_model = torch.nn.Sequential(
    torch.nn.Linear(1,2),
    torch.nn.Sigmoid(),
    torch.nn.Linear(2,1),
    torch.nn.Sigmoid()
)

yhat = seq_model(x)
print("yhat:", yhat)

yhat: tensor([0.5362], grad_fn=<SigmoidBackward0>)


Train a model

In [10]:
def train(Y, X, model, optimizer, criterion, epochs=100):
  cost = []
  total = 0
  for epoch in range(epochs):
    total = 0
    for x, y in zip(X, Y):
      yhat = model(x)
      loss = criterion(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total += loss.item()
    cost.append(total)
  return cost

In [11]:
model = NN(1,2,1)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

X = torch.arange(-20,20,1).view(-1,1).type(torch.FloatTensor)
Y = torch.zeros(X.shape[0],1)
Y[(X[:,0] > -4) & (X[:,0] < 4)] = 1.0

cost = train(Y, X, model, optimizer, criterion)

print("cost: ", cost)

cost:  [32.79397138953209, 29.330220818519592, 26.743085086345673, 24.81369984149933, 23.37019833922386, 22.283473938703537, 21.458833009004593, 20.827566623687744, 20.33993637561798, 19.959834039211273, 19.660893619060516, 19.423719629645348, 19.233920738101006, 19.0807316750288, 18.956027939915657, 18.85363006591797, 18.768799424171448, 18.69787621498108, 18.638016134500504, 18.58698983490467, 18.543047711253166, 18.504799231886864, 18.47114270925522, 18.441194266080856, 18.414247199892998, 18.389728918671608, 18.3671785145998, 18.34622000157833, 18.326547637581825, 18.307912096381187, 18.290108159184456, 18.272967740893364, 18.256351247429848, 18.24014486372471, 18.22425177693367, 18.208595260977745, 18.193107798695564, 18.177736192941666, 18.162433996796608, 18.147163167595863, 18.131891056895256, 18.11659114062786, 18.10124135017395, 18.085822597146034, 18.070319637656212, 18.054718926548958, 18.03900983929634, 18.023184150457382, 18.007234379649162, 17.991156190633774, 17.9749440

**More hidden neurons**

In [12]:
from torch.utils.data import Dataset, DataLoader

In [17]:
class Data(Dataset):
  def __init__(self):
    self.x = torch.linspace(-20,20,100).view(-1,1)
    self.y = torch.zeros(self.x.shape[0])
    self.y[(self.x[:,0] > -10 & (self.x[:,0] < -5))] = 1
    self.y[(self.x[:,0] > 5 & (self.x[:,0] < 10))] = 1
    self.y = self.y.view(-1,1)
    self.len = self.x.shape[0]

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.len

In [14]:
class NN(nn.Module):
  def __init__(self, D_in, H, D_out):
    super(NN, self).__init__()
    self.linear1 = nn.Linear(D_in, H)
    self.linear2 = nn.Linear(H, D_out)

  def forward(self, x):
    x = sigmoid(self.linear1(x))
    x = sigmoid(self.linear2(x))
    return x

In [15]:
def train(data_set, model, criterion, train_loader, optimizer, epochs=5, plot_number=10):
  cost = []
  for epoch in range(epochs):
    total = 0
    for x,y in train_loader:
      yhat = model(x)
      loss = criterion(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total += loss.item()
    cost.append(total)
  return cost

In [22]:
model = NN(1,6,1)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_dataset = Data()
train_loader = DataLoader(dataset=train_dataset, batch_size=100)

cost = train(train_dataset, model, criterion, train_loader, optimizer, epochs=100)

i = 0
for c in cost:
  i += 1
  if i % 10 == 0:
    print(c)

0.6289365887641907
0.5013830065727234
0.41113007068634033
0.3389420211315155
0.28094714879989624
0.23523074388504028
0.1994495987892151
0.17137759923934937
0.1491583287715912
0.1313517540693283


same as above, but using nn.Sequential

In [23]:
model = torch.nn.Sequential(
    torch.nn.Linear(1,6),
    torch.nn.Sigmoid(),
    torch.nn.Linear(6,1),
    torch.nn.Sigmoid()
)

**Multidimensional input data**

In [24]:
class XOR_Data(Dataset):

    # Constructor
    def __init__(self, N_s=100):
        self.x = torch.zeros((N_s, 2))
        self.y = torch.zeros((N_s, 1))
        for i in range(N_s // 4):
            self.x[i, :] = torch.Tensor([0.0, 0.0])
            self.y[i, 0] = torch.Tensor([0.0])

            self.x[i + N_s // 4, :] = torch.Tensor([0.0, 1.0])
            self.y[i + N_s // 4, 0] = torch.Tensor([1.0])

            self.x[i + N_s // 2, :] = torch.Tensor([1.0, 0.0])
            self.y[i + N_s // 2, 0] = torch.Tensor([1.0])

            self.x[i + 3 * N_s // 4, :] = torch.Tensor([1.0, 1.0])
            self.y[i + 3 * N_s // 4, 0] = torch.Tensor([0.0])

            self.x = self.x + 0.01 * torch.randn((N_s, 2))
        self.len = N_s

    # Getter
    def __getitem__(self, index):
        return self.x[index],self.y[index]

    # Get Length
    def __len__(self):
        return self.len

In [26]:
# Calculate accuracy

def accuracy(model, data_set):
    return np.mean(data_set.y.view(-1).numpy() == (model(data_set.x)[:, 0] > 0.5).numpy())

In [31]:
def train(dataset, model, criterion, train_loader, optimizer, epochs=100):
  cost = []
  acc = []
  for epoch in range(epochs):
    total = 0
    for x,y in train_loader:
      yhat = model(x)
      loss = criterion(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total = loss.item()
    acc.append(accuracy(model, dataset))
    cost.append(total)

  return cost, acc

In [32]:
model = NN(2,4,1)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
train_dataset = XOR_Data()
train_loader = DataLoader(dataset=train_dataset, batch_size=1)

cost, acc = train(train_dataset, model, criterion, train_loader, optimizer)

In [35]:
for i in range(len(cost)):
  if i % 10 == 0:
    print(f"cost: {cost[i]} - accuracy: {acc[i]}")

cost: 0.712531566619873 - accuracy: 0.5
cost: 0.6888075470924377 - accuracy: 0.25
cost: 0.6877162456512451 - accuracy: 0.25
cost: 0.6867204308509827 - accuracy: 0.25
cost: 0.6857218146324158 - accuracy: 0.49
cost: 0.6847257018089294 - accuracy: 0.5
cost: 0.6837389469146729 - accuracy: 0.5
cost: 0.6827698349952698 - accuracy: 0.5
cost: 0.6818262934684753 - accuracy: 0.5
cost: 0.680915892124176 - accuracy: 0.5
